# DAT-NYC-37 | Lab 07 | Introduction to Regression and Model Fit, Part 2 | Answer Key

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn import feature_selection, linear_model

pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

In [2]:
df = pd.read_csv(os.path.join('..', 'datasets', 'credit.csv'))

IOError: File ../datasets/credit.csv does not exist

In [ ]:
df.head()

A description of the dataset is as follows:

- Income (in thousands of dollars)
- Rating: Credit score rating
- Cards: Number of Credit cards owned
- Age
- Education: Years of Education
- Gender: Male/Female
- Student: Yes/No
- Married: Yes/No
- Ethnicity: African American/Asian/Caucasian
- Balance: Average credit card debt

> ## Question 1.  Let's explore the quantitative variables that affect `Balance`.  From your preliminary analysis, which 2 variables seem to affect `Balance` the most?  Our goal is interpretation; can we use these 2 variables simultaneously?  Why or why not?

In [ ]:
df.corr()

Answer: `Income` and `Rating` have the highest impact on `Balance`.  We cannot use these 2 variables simultaneously because they are highly correlated.

> ## Question 2.  `Race`, `Gender`, `Married`, and `Student` are categorical variables.  Go ahead and create dummy variables for all of them.

In [ ]:
race_df = pd.get_dummies(df.Ethnicity, prefix = 'Race')
gender_df = pd.get_dummies(df.Gender, prefix = 'Gender')
married_df = pd.get_dummies(df.Married, prefix = 'Married')
student_df = pd.get_dummies(df.Student, prefix = 'Student')

df = df.join([race_df, gender_df, married_df, student_df])

df.columns

> ## Question 3.  Using sklearn and a linear regression, predict `Balance` using `Income`, `Cards`, `Age`, `Education`, `Gender`, and `Race`

First, find the coefficients of your regression line.

In [ ]:
X = df[ ['Income', 'Cards', 'Age', 'Education', 'Gender_Male', 'Race_Asian', 'Race_Caucasian'] ]
y = df.Balance

model = linear_model.LinearRegression()
model.fit(X,y)

print model.intercept_
print model.coef_

Then, find the p-values of your estimates.  You have a few variables try to show your p-values alongside the names of the variables.

In [ ]:
zip(X.columns.values, feature_selection.f_regression(X, y)[1])

> ## Question 4.  Which of your coefficients are significant at the 5% significance level?

Answer: Only `Income` is.

> ## Question 5.  What is your model's $R^2$?

In [ ]:
model.score(X, y)

> ## Question 6.  How do we interpret this value?

Answer: 23% of the variability of `Balance` is captured by the linear model.

> ## Question 7.  Now let's focus on the two most significant variables from your previous model and re-run your regression model.

In [ ]:
X = df[ ['Income', 'Cards'] ]
y = df.Balance

model = linear_model.LinearRegression()
model.fit(X,y)

print model.intercept_
print model.coef_

> ## Question 8.  In comparison to the previous model, did the $R^2$ increase or decrease?  Why?

In [ ]:
model.score(X, y)

Answer: It decreased since we used subset of the variables we used earlier.  This does not mean that the precision of our model has decreased. $R^2$ on its own is not a good measure to compare two models.

> ## Question 9.  Now let's regress `Balance` on `Gender` alone.  After running your linear regressions, do you have enough evidence to claim that females have more balance than males?  (Hint: Look at the p-value of the Gender coefficient.  If it is significant then you will have evidence to support that claim, otherwise you cannot support the statement.)

In [ ]:
X = df[ ['Gender_Female'] ]
y = df.Balance

model = linear_model.LinearRegression()
model.fit(X,y)

print model.intercept_
print model.coef_
print X.columns.values, feature_selection.f_regression(X, y)[1]

Answer: (If your answer is yes, interpret the results).  The p-value is extremely high: Although the coefficient of dummy variable for females is positive, we cannot claim females spend more on average than males.

> ## Question 10.  Now let's regress `Balance` on `Ethnicity`.  After running your linear regressions, do you have enough evidence to claim that some ethnic groups carry more balance than others?

In [ ]:
X = df[ ['Race_Asian', 'Race_Caucasian'] ]
y = df.Balance

model = linear_model.LinearRegression()
model.fit(X,y)

print model.intercept_
print model.coef_
print feature_selection.f_regression(X, y)[1]

Answer: (If your answer is yes, interpret the results).  Again the p-values are extremely high.  We don't have enough evidence to say that an ethnic group carries more balance than others.

> ## Question 11.  Finally let's regress `Balance` on `Student`.  After running your linear regressions, do you have enough evidence to claim that students carry more balance than non-students?

In [ ]:
X = df[ ['Student_Yes'] ]
y = df.Balance

model = linear_model.LinearRegression()
model.fit(X,y)

print model.intercept_
print model.coef_
print feature_selection.f_regression(X, y)[1]

Answer: (If your answer is yes, interpret the results).  Yes, we have enough evidence to support that students are on average carrying \$396 more balance than non-students.

> ## Question 12.  No let's consider the effect of `Student` and `Income` on `Balance` simultaneously.  Are all the coefficients significant?

In [ ]:
X = df[ ['Student_Yes', 'Income'] ]
y = df.Balance

model = linear_model.LinearRegression()
model.fit(X,y)

print model.intercept_
print model.coef_
print feature_selection.f_regression(X, y)[1]

Answer: (If your answer is yes, interpret the results).  Yes, both coefficients are significant.  We find that when fixing income, students on average tend to carry \$382 more balance than non-students.  Also, on average higher income earners tend to carry more balance on their credit cards: For every \$1,000 additional income, people on average carry around a higher balance of \$6.

> ## Question 13.  No let's consider the interaction effect of `Student` and `Income` on `Balance` simultaneously.  Are all the coefficients significant?  It they are, write down your regression model below

(First generate a new variable for the interaction term)

In [ ]:
df['Student * Income'] = df['Student_Yes'] * df['Income']

In [ ]:
X = df[ ['Student_Yes', 'Income', 'Student * Income'] ]
y = df.Balance

model = linear_model.LinearRegression()
model.fit(X,y)

print model.intercept_
print model.coef_
print feature_selection.f_regression(X, y)[1]

Answer: Yes, they are all significant.  $Balance = 200 + 477 * Student_{Yes} + 6.21 * Income - 2 * Income * Student_{Yes}$

> ## Question 14.  Is there any income level at which students and non-students on average carry same level of balance?

Answer:

- Non-students: $Balance = 200 + 6.21 * Income$
- Students: $Balance = 200 + 477 + 6.21 * Income - 2 * Income$

At the \$238 income level, these two groups carry the same amount of balance.  Since this range of income is higher than any observed values for students, it is safe to say that within the range of our observations, students on average carry more balance.

We interpret the results this way: We say, students on average carry \$477 more than non-students.  But for every \$1,000 they make this difference between the balance that students and non-students are carrying is decreased by \$2.